In [124]:
import pandas as pd
import numpy as np

In [125]:
data = pd.read_csv(r'C:\Users\forma\Desktop\machine-learning-2c-2025\data.csv', low_memory=False, sep='|')

In [126]:
data.columns

Index(['client_id', 'Target', 'Month', 'First_product_dt', 'Last_product_dt',
       'CreditCard_Premium', 'CreditCard_Active', 'CreditCard_CoBranding',
       'Loan_Active', 'Mortgage_Active', 'SavingAccount_Active_ARG_Salary',
       'SavingAccount_Active_ARG', 'SavingAccount_Active_DOLLAR',
       'DebitCard_Active', 'Investment_Active', 'Package_Active',
       'Insurance_Life', 'Insurance_Home', 'Insurance_Accidents',
       'Insurance_Mobile', 'Insurance_ATM', 'Insurance_Unemployment', 'Sex',
       'Client_Age_grp', 'SavingAccount_Balance_FirstDate',
       'SavingAccount_Balance_LastDate', 'SavingAccount_Balance_Average',
       'SavingAccount_Days_with_use', 'SavingAccount_Days_with_Credits',
       'SavingAccount_Days_with_Debits',
       'SavingAccount_Salary_Payment_Transactions',
       'SavingAccount_Transfer_In_Transactions',
       'SavingAccount_ATM_Extraction_Transactions',
       'SavingAccount_Service_Payment_Transactions',
       'SavingAccount_CreditCard_Payment_T

In [127]:
print(data['client_id'].value_counts())

client_id
4221845                   9
7219504                   9
5882370                   9
1158013                   9
3914580                   9
                         ..
6623284                   1
672542                    1
264018                    1
4424661                   1
(238615 rows affected)    1
Name: count, Length: 26561, dtype: int64


In [139]:
# Busco los ID´s de los clientes que tienen 9 meses de data

# Creo una tabla con ID_cliente, cantidad de apariciones
ids_clientes_9m = data['client_id'].value_counts()

# Me quedo solo con aquellos que aparecen 9 veces 
ids_clientes_9m = ids_clientes_9m[ids_clientes_9m == 9].index

(26483,)

In [140]:
# [agosto, septiembre, octubre, noviembre, diciembre, enero] TW, [febrero] LW, [marzo, abril] PW

# me quedo con los clientes que en el ultimo mes de la TW tienen package_active = no & CreditCard_CoBranding = no, y ademas, aparecen en los 9 meses
filtered_bussiness_conditions = data_clientes_9m[(data_clientes_9m['Month'] == '2019-01-01') 
                    & (data_clientes_9m['Package_Active'] == 'No')
                    & (data_clientes_9m['CreditCard_CoBranding'] == 'No')]

# Obtengo sus id
ids_filtered_bussiness_conditions = filtered_bussiness_conditions['client_id'].values
ids_filtered_bussiness_conditions.shape

(23191,)

In [144]:
# Ahora unifico la tabla de clientes filtrados por paquete con los que aparecen 9 meses 

data_final = data[data['client_id'].isin(ids_filtered_bussiness_conditions)]
data_final.shape

(208719, 77)

In [145]:
# Ahora ya puedo separar las ventanas
training_window = data_final[data_final['Month'].isin(['2018-08-01',    
'2018-09-01',
'2018-10-01',
'2018-11-01',
'2018-12-01',
'2019-01-01'   
])]

training_window.shape

(139146, 77)

In [ ]:
data_lead_window = data_final[data_final['Month'].isin(['2018-08-01'])]

data_training_window.shape